In [ ]:
import pandas as pd
import numpy as np

In [51]:
import pandas as pd
import numpy as np
from datetime import datetime

# Define valid month abbreviations → full names
month_map = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}

regions_to_drop = [
    "Philippines",
    "NCR", "CAR", "I", "II", "III", "IV-A", "MIMAROPA", "V", "VI", "NIR",
    "VII", "VIII", "IX", "X", "XI", "XII", "XIII", "BARMM"
]
pattern = r"^(" + "|".join(regions_to_drop) + r")\b"

def phase_to_date(phase):
    phase = str(phase).strip()
    if not phase:
        return np.nan
    
    if phase.startswith("First Phase"):
        day = 15
    elif phase.startswith("Second Phase"):
        parts = phase.split()
        if len(parts) >= 3:
            month_abbr = parts[2].capitalize().strip('.,')
            # Use 28 if it's February (or Feb)
            if month_abbr == 'Feb':
                day = 28
            else:
                day = 30
        else:
            return np.nan
    else:
        return np.nan

    parts = phase.split()
    if len(parts) < 4:
        return np.nan

    month_abbr = parts[2].capitalize().strip('.,')
    year = parts[3].strip('.,')

    month = month_map.get(month_abbr, None)
    if not month:
        return np.nan

    try:
        date = datetime.strptime(f"{day} {month} {year}", "%d %B %Y")
        return date.strftime("%Y-%m-%d")
    except Exception as e:
        print(f"[phase_to_date] Parse fail: {phase} → {e}")
        return np.nan

def load_clean_transform(file_name, sheet_name, target_month_abbr, target_year):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=[4, 5])
    df.columns = [
        '_'.join([str(i).strip() for i in col if 'Unnamed' not in str(i)])
        for col in df.columns
    ]
    df = df[df['Region / Province'].notna()]
    df = df[~df['Region / Province'].astype(str).str.contains('phase', case=False)]
    df = df.iloc[:-6]

    melted = pd.melt(
        df,
        id_vars=['Region / Province'],
        var_name='Commodity_Phase',
        value_name='Price'
    )

    melted[['Commodity', 'Phase']] = melted['Commodity_Phase'].str.split('_', n=1, expand=True)
    melted = melted.drop(columns=['Commodity_Phase'])
    melted = melted[['Region / Province', 'Commodity', 'Phase', 'Price']]
    melted = melted.rename(columns={"Region / Province": "Province"})

    melted["Phase"] = (
        melted["Phase"]
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
        .str.replace(r"\.\d+$", "", regex=True)
    )

    melted["Date"] = melted["Phase"].apply(phase_to_date)

    # Filter only phases that match the target month/year (using 3-letter format)
    melted = melted[
        melted["Phase"].str.contains(f"{target_month_abbr} {target_year}", case=False, na=False)
    ]

    melted = melted[~melted["Province"].str.strip().str.match(pattern)]
    melted = melted[melted['Price'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]

    melted = melted[
    melted['Price'].apply(lambda x: str(x).replace('.', '', 1).isdigit())
    ]

    melted['Price'] = melted['Price'].astype(float)

    # Drop zero or negative prices
    melted = melted[melted['Price'] > 0]
    
    return melted

In [ ]:
sheets = {
    'rice': 'Table 1_rice',
    'meat': 'Table 2_meat',
    'veg': 'Table 4_veg',
    'fruits': 'Table 6_fruits'
}

files = [
    {"file": "dataset_jul24.xlsx", "month": "Jul", "year": "2024"},
    {"file": "dataset_aug24.xlsx", "month": "Aug", "year": "2024"},
    {"file": "dataset_sep24.xlsx", "month": "Sep", "year": "2024"},
    {"file": "dataset_oct24.xlsx", "month": "Oct", "year": "2024"},
    {"file": "dataset_nov24.xlsx", "month": "Nov", "year": "2024"},
    {"file": "dataset_dec24.xlsx", "month": "Dec", "year": "2024"},
    {"file": "dataset_jan25.xlsx", "month": "Jan", "year": "2025"},
    {"file": "dataset_feb25.xlsx", "month": "Feb", "year": "2025"},
    {"file": "dataset_mar25.xlsx", "month": "Mar", "year": "2025"},
    {"file": "dataset_apr25.xlsx", "month": "Apr", "year": "2025"},
    {"file": "dataset_may25.xlsx", "month": "May", "year": "2025"}
    {"file": "dataset_jun25.xlsx", "month": "Jun", "year": "2025"}
]

all_data = {}

for f in files:
    this_file = f["file"]
    month = f["month"]
    year = f["year"]
    all_data[this_file] = {
        key: load_clean_transform(this_file, sheet, month, year)
        for key, sheet in sheets.items()
    }

# Example:
df_meat_feb = all_data["dataset_feb25.xlsx"]['meat']
df_meat_feb["Phase"].unique()  # This will show the unique phases for meat in February 2025

array(['First Phase Feb 2025', 'Second Phase Feb 2025'], dtype=object)

In [ ]:
# To query specific dataframes for each commodity in the specified months, you can use the following code:
# all_data["dataset_monthyear.xlsx"]['commodity_name']
# Note that the corresponding file name should contain specifically 3 letters
df_meat_may = all_data["dataset_may25.xlsx"]['meat']
df_meat_jan = all_data["dataset_jan25.xlsx"]['meat']
df_meat_feb = all_data["dataset_feb25.xlsx"]['meat']
df_meat_march = all_data["dataset_march25.xlsx"]['meat']
df_meat_aug24 = all_data["dataset_august24.xlsx"]['meat']
df_meat_aug24[df_meat_aug24['Province'] == 'Quezon']  # This will show the meat prices for Quezon in August 2024

,Province,Commodity,Phase,Price,Date
263,Quezon,"BEEF, MEAT WITH BONES",First Phase Aug 2024,380.0,2024-08-15
491,Quezon,"BEEF, MEAT WITH BONES",Second Phase Aug 2024,380.0,2024-08-30
947,Quezon,"BEEF, PURE MEAT",First Phase Aug 2024,480.0,2024-08-15
1175,Quezon,"BEEF, PURE MEAT",Second Phase Aug 2024,480.0,2024-08-30
1631,Quezon,"PORK, KASIM",First Phase Aug 2024,330.0,2024-08-15
1859,Quezon,"PORK, KASIM",Second Phase Aug 2024,320.0,2024-08-30
2315,Quezon,"PORK, LIEMPO",First Phase Aug 2024,340.0,2024-08-15
2543,Quezon,"PORK, LIEMPO",Second Phase Aug 2024,330.0,2024-08-30
2999,Quezon,"PORK, MEAT WITH BONES",First Phase Aug 2024,320.0,2024-08-15
3227,Quezon,"PORK, MEAT WITH BONES",Second Phase Aug 2024,310.0,2024-08-30


In [57]:
# Merge all final dataframes into one big dataframe
all_data

{'dataset_august24.xlsx': {'rice':                Province             Commodity                  Phase  Price  \
  235                Abra         RICE, SPECIAL   First Phase Aug 2024  60.00   
  236              Apayao         RICE, SPECIAL   First Phase Aug 2024  52.80   
  237             Benguet         RICE, SPECIAL   First Phase Aug 2024  58.45   
  239             Kalinga         RICE, SPECIAL   First Phase Aug 2024  60.00   
  240   Mountain Province         RICE, SPECIAL   First Phase Aug 2024  66.50   
  ...                 ...                   ...                    ...    ...   
  1965     City of Butuan  RICE, REGULAR MILLED  Second Phase Aug 2024  51.40   
  1967            Basilan  RICE, REGULAR MILLED  Second Phase Aug 2024  53.00   
  1968      Lanao Del Sur  RICE, REGULAR MILLED  Second Phase Aug 2024  55.25   
  1969        Maguindanao  RICE, REGULAR MILLED  Second Phase Aug 2024  47.50   
  1971          Tawi-Tawi  RICE, REGULAR MILLED  Second Phase Aug 2024  51.0

,Region / Province,"BEEF, MEAT WITH BONES_Second Phase May 2025","BEEF, MEAT WITH BONES_Second Phase May 2025.1","BEEF, MEAT WITH BONES_First Phase Jun 2025","BEEF, MEAT WITH BONES_First Phase Jun 2025.1","BEEF, MEAT WITH BONES_Second Phase Jun 2025","BEEF, MEAT WITH BONES_Second Phase Jun 2025.1","BEEF, MEAT WITH BONES_Second Phase Jun 2025.2","BEEF, PURE MEAT_Second Phase May 2025","BEEF, PURE MEAT_Second Phase May 2025.1",...,"CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.10","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.11","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.12","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.13","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.14","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.15","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.16","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.17","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.18","CHICKEN EGG, MED. (per piece)_Second Phase Jun 2025.19"
2,Philippines,364.4,-,366.67,NaN,367.13,-,NaN,444.95,-,...,-,r,-,-,r,-,-,r,-,-
3,NCR- National Capital Region,414.45,-,415.22,NaN,416.19,-,NaN,470.41,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
5,CAR - Cordillera Administrative Region,379.07,-,380.88,NaN,382.36,-,NaN,392.63,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
6,Abra,300,-,300,NaN,300,-,NaN,300,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
7,Apayao,360,-,360,NaN,360,-,NaN,385,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Lanao Del Sur,340,-,340,NaN,340,-,NaN,-,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
132,Maguindanao,250,-,250,NaN,250,-,NaN,302.5,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
133,Sulu,285,-,285,NaN,275,-,NaN,-,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-
134,Tawi-Tawi,-,-,-,NaN,-,-,NaN,360,-,...,-,NaN,-,-,NaN,-,-,NaN,-,-


In [56]:
# Output to csv
melted_df.to_csv("retail_price_cleaned.csv", index=False)